In [ ]:
'''第二階段爬蟲'''
'''目標：針對所有具備 2017~2022 天氣資料的 NOAA 天氣觀測站點，送出 2017~2022 年天氣資料的.csv order'''
'''匯入套件'''
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException
# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait
# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC
# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By
# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 強制等待 (執行期間休息一下)
from time import sleep
# 整理 json 使用的工具
import json
# 執行 command 的時候用的
import os
# 美麗湯
from bs4 import BeautifulSoup as bs
# Regular Expression
import re

# 隨機取得 User-Agent
from fake_useragent import UserAgent
ua = UserAgent(use_external_data = True)

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")               #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  #禁用彈出攔截
my_options.add_argument("--disable-notifications")   #取消 chrome 推播通知
my_options.add_argument(f'--user-agent={ua.random}') #(Optional)加入 User-Agent

# 使用 Chrome 的 WebDriver
my_service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome(
    options = my_options,
    service = my_service
)

In [ ]:
'''讀取 JSON'''
# 讀取 NOAA.json
with open(f"C:/WBAN_url.json", "r", encoding="utf-8") as file:
    strJson = file.read()

# list 接 NOAA.json 的資訊
station = json.loads(strJson)

In [ ]:
'''下天氣資訊訂單'''
'''儲存訂單編號'''
# 建立 order list
lst_order = []

# 計數器
count = 0

# 運行設定區
# 本次初始值
start = 0
# 本次結束值(最大2464)
stop = 2464
# email
email = "e-mail"

# 迴圈下訂單
for i in station[start:stop]:
    '''判斷 page 1'''
    # 進入天氣觀測站頁面
    driver.get(i)

    # 睡一下
    sleep(3)

    # 取得檢視原始碼的內容
    html1 = driver.page_source

    # 指定 lxml 作為解析器
    soup1 = bs(html1, "lxml")

    # Regular Expression 判斷站點目前是否運作中
    match = re.search('>2023-', str(soup1.select('div.detailColLeft')))
    if not match:
        # 計數器 +1
        count += 1
        continue
    
    '''page 1'''
    while True:
        try:
            # 進入天氣觀測站頁面
            driver.get(i)

            # 睡一下
            sleep(3)

            # 取得檢視原始碼的內容
            html11 = driver.page_source

            # 指定 lxml 作為解析器
            soup11 = bs(html11, "lxml")

            # 等待元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.detailWhatNext.clearfix > a.button.cartButton')
                ))

            # 點選 "ADD to CHART" 按鈕
            driver.find_element(By.CSS_SELECTOR, 'div.detailWhatNext.clearfix > a.button.cartButton').click()

            # 睡一下
            sleep(5)

            # 取得檢視原始碼的內容
            html12 = driver.page_source

            # 指定 lxml 作為解析器
            soup12 = bs(html12, "lxml")

            # 確認訂單點選成功
            item = soup12.select('span#widgetBodyInner > div#widgetText')
            str_item = str(item)
            if str_item != '[<div id="widgetText">1 item</div>]':
                raise
            break
        except:
            continue

    '''page 2'''
    while True:
        try:
            # 進入購物車頁面
            driver.get("https://www.ncdc.noaa.gov/cdo-web/cart")
            
            # 睡一下
            sleep(3)
            
            # 取得檢視原始碼的內容
            html2 = driver.page_source

            # 指定 lxml 作為解析器
            soup2 = bs(html2, "lxml")

            # 點選 "選擇 CSV" 按鈕
            driver.find_element(By.CSS_SELECTOR, 'div.productSelection.clearfix > input#LCD_CUSTOM_CSV').click()

            # 點選 "日期欄位" 按鈕
            driver.find_element(By.CSS_SELECTOR, 'div#dateRangeContainer > input.noaa-daterange-input').click()

            # 等待元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.ui-datepicker-title > select.ui-datepicker-year')
                ))

            # 點選 "year" 按鈕
            yy = Select(driver.find_element(By.CSS_SELECTOR, 'div.ui-datepicker-title > select.ui-datepicker-year'))

            # 選擇年分
            yy.select_by_visible_text('2017')

            # 點選 "date" 按鈕
            driver.find_element(By.CSS_SELECTOR, 'a[data-date = "1"]').click()

            # 等待元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'button.noaa-daterange-btn.noaa-daterange-applybtn')
                ))

            # 點選 "APPLY" 按鈕
            driver.find_element(By.CSS_SELECTOR, 'button.noaa-daterange-btn.noaa-daterange-applybtn').click()

            # 等待元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'button.button.cartButton.floatRight')
                ))

            # 點選 "CONTINUE" 按鈕
            driver.find_element(By.CSS_SELECTOR, 'button.button.cartButton.floatRight').click()
            break
        except:
            continue

    '''page 3'''
    while True:
        try:
            # 睡一下
            sleep(3)
            
            # 取得目前網址
            now = driver.current_url

            # 取得檢視原始碼的內容
            html3 = driver.page_source

            # 指定 lxml 作為解析器
            soup3 = bs(html3, "lxml")

            # 等待元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.col-2-left-600 > div.boxblock.clearfix > input#email')
                ))

            # 尋找 email 輸入框
            email = driver.find_element(By.CSS_SELECTOR, 'div.col-2-left-600 > div.boxblock.clearfix > input#email')

            # 輸入 email
            email.send_keys(email)

            # 等待元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.col-2-left-600 > div.boxblock.clearfix > input#emailConfirmation')
                ))

            # 尋找 emailConfirmation 輸入框
            emailConfirmation = driver.find_element(By.CSS_SELECTOR, 'div.col-2-left-600 > div.boxblock.clearfix > input#emailConfirmation')

            # 輸入 emailConfirmation
            emailConfirmation.send_keys(email)
            
            # 取得檢視原始碼的內容
            html31 = driver.page_source

            # 指定 lxml 作為解析器
            soup31 = bs(html31, "lxml")

            # 等待元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.clearfix > input.button.cartButton.floatRight')
                ))
            
            # 點選 "SUBMIT ORDER" 按鈕
            driver.find_element(By.CSS_SELECTOR, 'div.clearfix > input.button.cartButton.floatRight').click()
            
            # 睡一下
            sleep(5)
            
            # 取得目前網址
            new = driver.current_url
            
            # 判斷是否跳轉
            if now == new:
                continue
            else:
                break
        except:
            driver.refresh()
            continue
    '''page 4'''
    # 取得檢視原始碼的內容
    html4 = driver.page_source

    # 指定 lxml 作為解析器
    soup4 = bs(html4, "lxml")

    # Regular Expression
    match = re.search('\d{7}', str(soup4.select('div#receipt')))

    # 生成 order list
    lst_order += [match[0]]

    # 計數器 +1
    count += 1
    
    # 睡一下
    sleep(3)

In [ ]:
'''JSON 轉換'''
with open(f"C:/ORDER_{start + count - 1}.json", "w", encoding="utf-8") as file:
    file.write(json.dumps(lst_order, ensure_ascii=False))

In [ ]:
'''運行狀態'''
print(f"清單完成了 {count} 次，下批訂單從 {start + count} 開始")
print(f"本次成功下了 {len(lst_order)} 次訂單")